In [46]:
import requests # library to handle requests
import bs4
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import numpy as np


CLIENT_ID = 'OOL3CJVWOLDGNU01D1VMCWKV2M1MQN5EJVXO13BBHWZPKGYR' # your Foursquare ID
CLIENT_SECRET = 'VZWPT2ARYG2CTCL1QSUTIUM2B5MFIIH4HOEJUGSUN15CXM3U' # your Foursquare Secret
ACCESS_TOKEN = 'KEZAPAWX2SID04SJG51EVSH5332ZQX1OXTFXMNEIYPWB2TBU' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100 # A default Foursquare API limit value

In [47]:
#Scrape the wikipeadia page using requests and BeautifulSoup
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
soup = bs4.BeautifulSoup(results.text, 'lxml')

#Create an array for extract the relevant using information
PostalCode = []
Borough = []
Neighborhood = []

#Extract elements from the table 
table = soup.find("table",class_ =  "wikitable sortable")


#convert elemnts into string
table = table.get_text()

#convert the elemnts of the table from string to list and split the list based on \n
table = table.split('\n')

#Remove white space from the list
str_list = list(filter(None, table))

In [48]:
# Extract PostalCode from the list
PostalCode = str_list[0:len(str_list):3]

# Extract Borough from the list
Borough = str_list[1:len(str_list):3]

# Extract Neighborhood from the list
Neighborhood = str_list[2:len(str_list):3]

# Convert list into Series
se1 = pd.Series(PostalCode)
se2 = pd.Series(Borough)
se3 = pd.Series(Neighborhood)

# Create a dataframe and add Series to the dataframe
Dataframe = pd.concat([se1,se2,se3], axis=1)

# Rename the columns 
Dataframe.columns = ["PostalCode", "Borough", "Neighborhood"]


In [49]:
# Remove "Not Assigned" within the column "Borough"
df = Dataframe[Dataframe["Borough"] != "Not assigned"].reset_index(drop=True)

# Remove the first row as it contains the column name of the table
df = df.drop(df.index[0])

In [50]:
# Open CSV file and convert it into a new dataframe named locgeo_df
locgeo_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')

# Merge dataframe df with locgeo_df
toronto_data = df.join(locgeo_df,  on='PostalCode')

In [51]:
# Include only Borough that contain the word Toronto
toronto_data = toronto_data[toronto_data["Borough"].str.contains("Toronto")].reset_index(drop=True)

In [52]:
#Function to repeat the same process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:

toronto_venues = getNearbyVenues(toronto_data['Neighborhood'], toronto_data['Latitude'], toronto_data['Longitude'])


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Har

In [54]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep = '')

# add Toronto_neighborhood column back to dataframe
toronto_onehot['Toronto_Neighborhood'] = toronto_venues['Neighborhood']
toronto_onehot.rename(columns={'Neighborhood':'Neighborhood_Venue' })

# move Toronto_neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Toronto_Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# group rows by Toronto_Neighborhood and by taking the mean of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Toronto_Neighborhood').mean().reset_index()
toronto_grouped

,Toronto_Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.016949,0.00000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.1875,0.125,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.016667,0.00000,0.000000,0.016667,0.016667
5,Christie,0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012658,0.0000,0.0000,0.0000,0.0000,0.000,0.012658,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.025316
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.030000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.00000,0.000000,0.010000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.029412,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000


In [56]:
# Function to sort the venue in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
# Create new dataframe to display the top 10 venue of each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Toronto_Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Toronto_Neighborhood'] = toronto_grouped['Toronto_Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()



,Toronto_Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Beer Bar,Restaurant,Farmers Market,Cheese Shop,Seafood Restaurant,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Coffee Shop,Breakfast Spot,Burrito Place,Convenience Store,Restaurant,Italian Restaurant,Stadium,Intersection
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Park,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Light Rail Station
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Harbor / Marina,Bar,Plane,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Boutique
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Thai Restaurant,Salad Place,Ramen Restaurant


In [58]:
# Set number of clusters
Kclusters = 5

# RDrop the column Toronto_Neighborhood in order to perfrom KMeans clustering to the dataframe
toronto_group_clustering = toronto_grouped.drop('Toronto_Neighborhood', 1)

# Run K Means clustering
Kmeans = KMeans(n_clusters = Kclusters, random_state = 0).fit(toronto_group_clustering)

In [59]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', Kmeans.labels_)
toronto_merged = toronto_data


# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Toronto_Neighborhood'), on='Neighborhood')

toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [60]:
# Display toronto_merged
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Cosmetics Shop,Shoe Store,Brewery
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,College Auditorium,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Café,Hotel,Cosmetics Shop,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Gastropub,Cocktail Bar,Gym,Moroccan Restaurant,Department Store,Lingerie Store,Clothing Store,Park
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Pub,Trail,Neighborhood,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Beer Bar,Restaurant,Farmers Market,Cheese Shop,Seafood Restaurant,Breakfast Spot
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Thai Restaurant,Salad Place,Ramen Restaurant
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Baby Store,Restaurant,Italian Restaurant,Nightclub,Candy Store,Athletics & Sports,Bank
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Bakery,Deli / Bodega,Gym,Thai Restaurant,Seafood Restaurant
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Pharmacy,Bakery,Grocery Store,Furniture / Home Store,Café,Middle Eastern Restaurant,Bar,Bank,Supermarket,Pool


In [61]:
# Use geopy library to get the latitude and longtitude values of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [62]:
# create map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(Kclusters)
ys = [i + x + (i*x)**2 for i in range(Kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
.